In [ ]:
import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go
import pickle
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
# this might give errors if you don't have newer ver sof prophet. in that case, comment it out and load using pickle
from fbprophet.serialize import model_to_json, model_from_json  
import json
from sklearn import metrics

In [ ]:
# Code to load pretraines model with pickle
with open("pickled_model", "rb") as f:
   m = pickle.load(f)
   
# Code to load model with fb prophet serialize if that works
# with open('serialized_model.json', 'r') as fin:
#     m = model_from_json(json.load(fin))  # Load model

In [ ]:
# Reading in data
data = pd.read_csv("../input/msa-merged-data/msa_merged_data.csv")
data["BEGIN_DATE_GMT"] = pd.to_datetime(data["BEGIN_DATE_GMT"])

# X-y split
y = data["AIL_DEMAND"]
X = data.drop(columns = ["AIL_DEMAND"])

In [ ]:
# you'll probabs need to modify these quite abit, because we are using only the temperature of 2019 or whatever year, not the actual future temperture + the model is lagged 

# Functions to make FBProphet Datasets
def make_prophet_df(X, y, regressors):
    data = pd.DataFrame()
    data["y"] = y
    data['ds'] = X["BEGIN_DATE_GMT"]
    for i in regressors:
        data[i] = X[i]
    return data

def make_future_df(prophet_model, df_train, df_test, include_history = True):
    # Creating the dataframe with datetime values to predict on (making predictions on train as well as the test set)
    future_dates = prophet_model.make_future_dataframe(periods=len(df_test), freq='H', include_history= include_history)
    # Adding regressors 
    if include_history:
        future_dates = pd.merge(future_dates, (df_train.append(df_test)).drop('y', axis=1), on = 'ds')
    else:
        future_dates = pd.merge(future_dates, df_test.drop('y', axis=1), on = 'ds')

    return future_dates

# Making Lags
# Choose Regressors
X["Calgary_temp.1_hour_lag"]=X["Calgary_temp"].shift(1)
X["Edmonton_temp.1_hour_lag"]=X["Edmonton_temp"].shift(1)
X["FortMM_temp.1_hour_lag"]=X["FortMM_Temp"].shift(1)
X["Lethbridge_temp.1_hour_lag"]=X["Lethbridge_temp"].shift(1)
regressors = ["Calgary_temp.1_hour_lag","Edmonton_temp.1_hour_lag","FortMM_temp.1_hour_lag","Lethbridge_temp.1_hour_lag","future 1","WTI spot","workingday"]